In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from ultralytics import YOLO
from collections import defaultdict

In [ ]:
RUNS_FOLDER = "../../runs/segment"
PLOTS_DIR = "plots"
os.makedirs(PLOTS_DIR, exist_ok=True)
DATA_DIR = "../../data/data-seg.yaml"

In [ ]:
# noda: no data augmentation
yolov8s_seg_noda = os.path.join(RUNS_FOLDER, "yolov8s_seg_noda")
yolov8m_seg_noda = os.path.join(RUNS_FOLDER, "yolov8m_seg_noda")
yolov8l_seg_noda = os.path.join(RUNS_FOLDER, "yolov8l_seg_noda")
yolov11s_seg_noda = os.path.join(RUNS_FOLDER, "yolov11s_seg_noda")
yolov11m_seg_noda = os.path.join(RUNS_FOLDER, "yolov11m_seg_noda")
yolov11l_seg_noda = os.path.join(RUNS_FOLDER, "yolov11l_seg_noda")

models_noda = {
    "yolov8s": yolov8s_seg_noda,
    "yolov8m": yolov8m_seg_noda,
    "yolov8l": yolov8l_seg_noda,
    "yolov11s": yolov11s_seg_noda,
    "yolov11m": yolov11m_seg_noda,
    "yolov11l": yolov11l_seg_noda,
}

In [ ]:
yolov8m_hsv = os.path.join(RUNS_FOLDER, "yolov8m_hsv")
yolov8m_translate = os.path.join(RUNS_FOLDER, "yolov8m_translate")
yolov8m_hsvtranslate = os.path.join(RUNS_FOLDER, "yolov8m_hsvtranslate")
yolov8m_fliplr = os.path.join(RUNS_FOLDER, "yolov8m_fliplr")
yolov8m_30epochs_noda = os.path.join(RUNS_FOLDER, "yolov8m_30epochs_noda")
yolov8m_allda = os.path.join(RUNS_FOLDER, "yolov8m_allda")

models_da = {
    "yolov8m_hsv": yolov8m_hsv,
    "yolov8m_translate": yolov8m_translate,
    "yolov8m_hsvtranslate": yolov8m_hsvtranslate,
    "yolov8m_fliplr": yolov8m_fliplr,
    "yolov8m_30epochs_noda": yolov8m_30epochs_noda,
    "yolov8m_all": yolov8m_allda,
}

In [ ]:
yolov8s_hsv = os.path.join(RUNS_FOLDER, "yolov8s_hsv")
yolov8s_translate = os.path.join(RUNS_FOLDER, "yolov8s_translate")
yolov8s_scale = os.path.join(RUNS_FOLDER, "yolov8s_scale")
yolov8s_fliplr = os.path.join(RUNS_FOLDER, "yolov8s_fliplr")
yolov8s_no_aug = os.path.join(RUNS_FOLDER, "yolov8s_no_aug")
yolov8s_all_aug = os.path.join(RUNS_FOLDER, "yolov8s_all_aug")


models_da_s = {
    "yolov8s_hsv": yolov8s_hsv,
    "yolov8s_translate": yolov8s_translate,
    "yolov8s_scale": yolov8s_scale,
    "yolov8s_fliplr": yolov8s_fliplr,
    "yolov8s_all_aug": yolov8s_all_aug,
    "yolov8s_no_aug": yolov8s_no_aug,
}

In [ ]:
######################
###### VAL DATA ######
######################
# 75 images, 523 instances, 3 classes (deal, product_category_deal, sonstiges)
# Deal: 69 images, 512 instances
# Product: 5 images, 7 instances
# Sonstiges: 4 image, 4 instances

In [ ]:
data = defaultdict(list)

for model_name, model_dir in models_noda.items():
    model = YOLO(os.path.join(model_dir, "weights", "best.pt"))
    results = pd.read_csv(os.path.join(model_dir, "results.csv"))
    metrics = model.val(data=DATA_DIR)
    data["model"].append(model_name)
    for k, v in metrics.results_dict.items():
        data[k.replace("metrics/", "")].append(round(float(v) * 100, 0))
    data["speed_preprocess"].append(round(metrics.speed["preprocess"], 2))
    data["speed_inference"].append(round(metrics.speed["inference"], 2))
    data["speed_postprocess"].append(round(metrics.speed["postprocess"], 2))
    data["map50_95_per_class"].append([round(float(v) * 100, 0) for v in metrics.box.maps])

    data["train/box_loss"].append(results["train/box_loss"].to_list())
    data["train/seg_loss"].append(results["train/seg_loss"].to_list())
    data["train/cls_loss"].append(results["train/cls_loss"].to_list())
    data["val/box_loss"].append(results["val/box_loss"].to_list())
    data["val/seg_loss"].append(results["val/seg_loss"].to_list())
    data["val/cls_loss"].append(results["val/cls_loss"].to_list())
data = pd.DataFrame(data)
data.to_csv("evaluation/metrics_seg_noda.csv", index=False)

In [ ]:
data_da = defaultdict(list)

for model_name, model_dir in models_da.items():
    model = YOLO(os.path.join(model_dir, "weights", "best.pt"))
    results = pd.read_csv(os.path.join(model_dir, "results.csv"))
    metrics = model.val(data=DATA_DIR)
    data_da["model"].append(model_name)
    for k, v in metrics.results_dict.items():
        data_da[k.replace("metrics/", "")].append(round(float(v) * 100, 0))
    data_da["speed_preprocess"].append(round(metrics.speed["preprocess"], 2))
    data_da["speed_inference"].append(round(metrics.speed["inference"], 2))
    data_da["speed_postprocess"].append(round(metrics.speed["postprocess"], 2))
    data_da["map50_95_per_class"].append([round(float(v) * 100, 0) for v in metrics.box.maps])

    data_da["train/box_loss"].append(results["train/box_loss"].to_list())
    data_da["train/seg_loss"].append(results["train/seg_loss"].to_list())
    data_da["train/cls_loss"].append(results["train/cls_loss"].to_list())
    data_da["val/box_loss"].append(results["val/box_loss"].to_list())
    data_da["val/seg_loss"].append(results["val/seg_loss"].to_list())
    data_da["val/cls_loss"].append(results["val/cls_loss"].to_list())

data_da = pd.DataFrame(data_da)
data_da.to_csv("evaluation/metrics_seg_da.csv", index=False)

In [ ]:
data_da = defaultdict(list)

for model_name, model_dir in models_da_s.items():
    model = YOLO(os.path.join(model_dir, "weights", "best.pt"))
    results = pd.read_csv(os.path.join(model_dir, "results.csv"))
    metrics = model.val(data=DATA_DIR)
    data_da["model"].append(model_name)
    for k, v in metrics.results_dict.items():
        data_da[k.replace("metrics/", "")].append(round(float(v) * 100, 0))
    data_da["speed_preprocess"].append(round(metrics.speed["preprocess"], 2))
    data_da["speed_inference"].append(round(metrics.speed["inference"], 2))
    data_da["speed_postprocess"].append(round(metrics.speed["postprocess"], 2))
    data_da["map50_95_per_class"].append([round(float(v) * 100, 0) for v in metrics.box.maps])

    data_da["train/box_loss"].append(results["train/box_loss"].to_list())
    data_da["train/seg_loss"].append(results["train/seg_loss"].to_list())
    data_da["train/cls_loss"].append(results["train/cls_loss"].to_list())
    data_da["val/box_loss"].append(results["val/box_loss"].to_list())
    data_da["val/seg_loss"].append(results["val/seg_loss"].to_list())
    data_da["val/cls_loss"].append(results["val/cls_loss"].to_list())

data_da = pd.DataFrame(data_da)
data_da.to_csv("evaluation/metrics_seg_da_8s.csv", index=False)